##final

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [0]:
data_train.shape,data_test.shape

((147443, 12), (49148, 11))

In [0]:
data_train_copy = data_train.copy(deep = True)
data_test_copy = data_test.copy(deep = True)

In [0]:
def clean(dataset):
    data = dataset.copy(deep = True)
    
    data.Inspection_Results.replace({'E': 1,'B':2,'F' :3,'G' : 4,'D': 5, 'C':6 ,'A':7}, inplace=True) # add main
    
    from sklearn.impute import KNNImputer             # add main
    imputer = KNNImputer(n_neighbors=5)
    array = imputer.fit_transform(data)
    
    data = pd.DataFrame(data = array, columns= data.columns, index = data.index)  # add main
    data = np.round(data)  # add main
    
    data = data.drop(columns= ['LicenseNo'])   #add main
    data['Reason'] = data['Reason'].astype('object')
    data['RiskLevel'] = data['RiskLevel'].astype('object')

    return data

In [0]:
data_train.Inspection_Results.replace({'E': 1,'B':2,'F' :3,'G' : 4,'D': 5, 'C':6 ,'A':7}, inplace=True) # add main

In [0]:
X = data_train.drop(columns='Inspection_Results')
y = data_train.Inspection_Results

In [0]:
from sklearn.impute import KNNImputer             # add main
imputer = KNNImputer(n_neighbors=3)
train_array = imputer.fit_transform(X)
test_array = imputer.transform(data_test)
    
data_train_im = pd.DataFrame(data = train_array, columns= X.columns, index = X.index)  # add main
X = np.round(data_train_im)  # add main

data_test_im = pd.DataFrame(data = test_array, columns= data_test.columns, index = data_test.index)  # add main
test_data = np.round(data_test_im)  # add main

In [0]:
def clean(dataset):
    data = dataset.copy(deep = True)
    
    data = data.drop(columns= ['LicenseNo'])   #add main
    data['Reason'] = data['Reason'].astype('object')
    data['RiskLevel'] = data['RiskLevel'].astype('object')

    return data

In [0]:
data_raw = pd.get_dummies(data_train )   ## add main

In [0]:
X_ = clean(X)
data_test_c = clean(test_data)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc,recall_score,classification_report

X_train,X_val, y_train,y_val = train_test_split(X_,y, train_size = .8, stratify = y, random_state = 6)

In [0]:
X_train.shape, y_train.shape

((117954, 10), (117954,))

In [0]:
model =RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,       ##add main
                       criterion='entropy', max_depth=10, max_features=10,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=9,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=61,
                       verbose=0, warm_start=False)

model.fit(X_train, y_train)

y_predict = model.predict(X_val)
print(confusion_matrix(y_val,y_predict)) 
print(accuracy_score(y_val,y_predict))
print(classification_report(y_val,y_predict))


In [0]:
data_test_copy['Inspection_Results'] = model.predict(data_test_c)

In [0]:
data_test_copy[['LicenseNo','Inspection_Results']].to_csv('rgd_food_rf.csv')